In [ ]:
import sys
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image as keras_image
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19
from keras.models import Model, model_from_json
from keras.optimizers import SGD

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
IMAGES_PATH = "../inputs/images/"
LABELS_PATH = "../inputs/labels/"

WEIGHTS_PATH = "../outputs/weights/"
NUM_CLASSES  = 8 

In [ ]:
def read_inputs():
    X = []
    Y = []
    for root, dirs, files, in os.walk(LABELS_PATH):
        for file in files:
            if file.endswith(".txt") and not file.startswith("."):
                images_path = root.replace(LABELS_PATH, IMAGES_PATH)
                images      = list(filter(lambda x: not x.startswith("."), os.listdir(images_path))) # keep images that doesn't start with .

                label_file = open(os.path.join(root,file),"r")
                label = int(float(label_file.readline().strip("\n")))

                images.sort() # only get the last four (maybe ?)
                for img in images[-4:]:
                    tmp = keras_image.load_img(images_path + "/" + img , target_size=(64, 64))
                    tmp = keras_image.img_to_array(tmp) / float(255)
                    X.append(tmp)
                    Y.append(label)
    return np.asarray(X), np_utils.to_categorical(np.asarray(Y), NUM_CLASSES)

In [ ]:
def simple_model():
    model = Sequential()
    model.add(Flatten(input_shape=(64, 64, 3)))
    model.add(Dense(units=64, activation='relu'))
    model.add(Dense(units=8, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    return model

In [ ]:
def VGGNET16():
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape = (64,64,3)) 
    
    stacked = Flatten()(vgg16.output)
    stacked = Dense(64, activation='relu')(stacked)
    predictions = Dense(8, activation='softmax')(stacked)
    model = Model(input=vgg16.input, output=predictions)
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model  

In [ ]:
X, Y = read_inputs()

In [ ]:
model = VGGNET16()

In [ ]:
#checkpoint = ModelCheckpoint(WEIGHTS_PATH + "vgg16-weights.h5", verbose=1, save_best_only=True)
#callbacks_list = [checkpoint]
model.fit(X, Y, epochs=200, validation_split=0.15, verbose=1, shuffle=True)

In [ ]:
plt.imshow(X[0])

In [ ]:
preds = model.predict(X[:10])

In [ ]:
print(preds[0])

In [ ]:
print(Y[0])